# **Deployment using Django :**

In [ ]:
!pip install django==4.2 pyngrok==6.0.0 torch torchvision opencv-python timm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 83.8 MB/s eta 0:00:00
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=16264fdf1d16c4508939c6b8d8f4669b85def4b5af5d74d9166ef43f9b263449
  Stored in directory: /root/.cache/pip/wheels/26/69/f6/58a89ce2f2ca23ffe8ae3fad678ebe8e1354f8cba1cfdef063
Successfully built pyngrok
  Attempting uninstall: pyngrok
    Found existing installation: pyngrok 7.1.6
    Uninstalling pyngrok-7.1.6:
      Successfully uninstalled pyngrok-7.1.6
  Attempting uninstall: django
    Found existing installation: Django 5.0.6
    Uninstalling Django-5.0.6:
      Successfully uninstalled Django-5.0.6


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MODEL_PATH = "/content/drive/MyDrive/shoplifter_model.pth"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Create a new Django project
# ============================================================
!django-admin startproject shoplifter_project
%cd shoplifter_project

# ============================================================
# Modify settings to allow all hosts
# ============================================================
settings_path = "shoplifter_project/settings.py"
with open(settings_path, "r") as file:
    settings = file.read()

settings = settings.replace("ALLOWED_HOSTS = []", "ALLOWED_HOSTS = ['*']")

with open(settings_path, "w") as file:
    file.write(settings)

# ============================================================
# Create a new application for Detection
# ============================================================
!python manage.py startapp detection_app

# Add the application to the settings
with open(settings_path, "a") as file:
    file.write("\nINSTALLED_APPS += ['detection_app']\n")

# ============================================================
# Professional HTML interface design
# ============================================================
import os
os.makedirs("detection_app/templates", exist_ok=True)

html_code = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Shoplifting Detection</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #0f172a;
            color: #fff;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            flex-direction: column;
        }
        .container {
            background-color: #1e293b;
            padding: 30px;
            border-radius: 15px;
            box-shadow: 0 8px 20px rgba(0,0,0,0.4);
            width: 400px;
            text-align: center;
        }
        h1 {
            margin-bottom: 20px;
        }
        input[type="file"] {
            margin: 15px 0;
        }
        button {
            background-color: #3b82f6;
            color: #fff;
            border: none;
            padding: 10px 15px;
            border-radius: 10px;
            cursor: pointer;
        }
        button:hover {
            background-color: #2563eb;
        }
        .result {
            margin-top: 20px;
            font-size: 18px;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Shoplifting Detection</h1>
        <form method="POST" enctype="multipart/form-data">
            {% csrf_token %}
            <input type="file" name="video" accept="video/mp4" required>
            <br>
            <button type="submit">Upload & Predict</button>
        </form>
        {% if result %}
        <div class="result">
            <strong>Result:</strong> {{ result }}
        </div>
        {% endif %}
    </div>
</body>
</html>
"""

with open("detection_app/templates/index.html", "w") as file:
    file.write(html_code)



/content/shoplifter_project


In [ ]:
# Edit views.py to load and run the model.
# ============================================================
views_code = f"""
from django.shortcuts import render
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import cv2
import numpy as np
import os
from timm import create_model

# ================================
# Download the model
# ================================
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class EfficientNetB0_LSTM(nn.Module):
    def __init__(self, hidden_size=128, num_classes=1):
        super().__init__()
        self.backbone = create_model("efficientnet_b0", pretrained=False)
        self.backbone.classifier = nn.Identity()
        for param in self.backbone.parameters():
            param.requires_grad = False
        self.lstm = nn.LSTM(input_size=1280, hidden_size=hidden_size, batch_first=True)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        b, t, c, h, w = x.shape
        x = x.view(b * t, c, h, w)
        feats = self.backbone(x)
        feats = feats.view(b, t, -1)
        _, (h_n, _) = self.lstm(feats)
        h_n = h_n.squeeze(0)
        out = self.dropout(h_n)
        out = self.fc(out)
        return torch.sigmoid(out)

# Download the trained model
MODEL_PATH = r"{MODEL_PATH}"
model = EfficientNetB0_LSTM().to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE), strict=False)
model.eval()

# ================================
# Video processing and prediction function
# ================================
def process_video(video_path):
    IMG_SIZE = 224
    MAX_SEQ_LENGTH = 16

    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        frames.append(frame)
    cap.release()

    total = len(frames)
    if total >= MAX_SEQ_LENGTH:
        indices = np.linspace(0, total - 1, MAX_SEQ_LENGTH, dtype=int)
        frames = [frames[i] for i in indices]
    else:
        frames += [frames[-1]] * (MAX_SEQ_LENGTH - total)

    frames = np.array(frames, dtype=np.float32) / 255.0
    frames = torch.tensor(frames).permute(0, 3, 1, 2).unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        output = model(frames)
        prediction = (output > 0.5).int().item()

    return "🚨 Shoplifter Detected!" if prediction == 1 else "✅ No Shoplifter"

# ================================
# Website interface
# ================================
def home(request):
    result = None
    if request.method == "POST" and request.FILES.get("video"):
        video_file = request.FILES["video"]
        video_path = os.path.join("detection_app", "temp_video.mp4")
        with open(video_path, "wb+") as dest:
            for chunk in video_file.chunks():
                dest.write(chunk)
        result = process_video(video_path)
        os.remove(video_path)
    return render(request, "index.html", {{ "result": result }})
"""

with open("detection_app/views.py", "w") as file:
    file.write(views_code)


# Edit urls.py
urls_code = """
from django.urls import path
from detection_app.views import home

urlpatterns = [
    path('', home, name='home'),
]
"""

with open("shoplifter_project/urls.py", "w") as file:
    file.write(urls_code)


In [ ]:
# Running Django with ngrok

!ngrok authtoken "2ziogsZnboFUwEDalhfVoIXNHYg_2SqEcYMcg8dVSWVT8qoyM"
from pyngrok import ngrok

public_url = ngrok.connect(8000)
print("🌐 Public URL:", public_url)

!python manage.py runserver 0.0.0.0:8000

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


🌐 Public URL: NgrokTunnel: "https://ce88e33d125b.ngrok-free.app" -> "http://localhost:8000"
Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).

You have 18 unapplied migration(s). Your project may not work properly until you apply the migrations for app(s): admin, auth, contenttypes, sessions.
Run 'python manage.py migrate' to apply them.
August 23, 2025 - 21:44:19
Django version 4.2, using settings 'shoplifter_project.settings'
Starting development server at http://0.0.0.0:8000/
Quit the server with CONTROL-C.

[23/Aug/2025 21:44:25] "GET / HTTP/1.1" 200 1712
Not Found: /favicon.ico
[23/Aug/2025 21:44:25] "GET /favicon.ico HTTP/1.1" 404 2142
Forbidden (Origin checking failed - https://ce88e33d125b.ngrok-free.app does not match any trusted origins.): /
[23/Aug/2025 21:45:47] "POST / HTTP/1.1" 403 2583


# **Deployment Locally :**

In [ ]:
!pip install torch torchvision opencv-python-headless
!pip install efficientnet-pytorch
!pip install ipywidgets


  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16426 sha256=fc6c1bde69a1ca5c7226ea5c79f28c2832c4f76d5949f403aaa34e9dbe4549eb
  Stored in directory: /root/.cache/pip/wheels/9c/3f/43/e6271c7026fe08c185da2be23c98c8e87477d3db63f41f32ad
Successfully built efficientnet-pytorch
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.4 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import cv2
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.models import efficientnet_b0
import time
import os
from pathlib import Path
from IPython.display import display, HTML, clear_output
from ipywidgets import widgets, interact, Layout
import io
from PIL import Image
import base64

In [ ]:
print(f"PyTorch version: {torch.__version__}")

PyTorch version: 2.8.0+cu126


In [ ]:
# Model Definition
class EfficientNetB0_LSTM(nn.Module):
    """
    EfficientNet-B0 + LSTM model for video classification
    """
    def __init__(self, hidden_size=128, num_classes=1):
        super().__init__()
        # Use EfficientNet-B0 as backbone
        backbone = efficientnet_b0(pretrained=True)
        # Remove the final classification layer
        backbone.classifier = nn.Identity()
        self.backbone = backbone

        # Freeze backbone parameters for faster training/inference
        for param in self.backbone.parameters():
            param.requires_grad = False

        # LSTM for temporal modeling
        self.lstm = nn.LSTM(input_size=1280, hidden_size=hidden_size, batch_first=True)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # x shape: (batch_size, sequence_length, channels, height, width)
        b, t, c, h, w = x.shape

        # Reshape to process all frames at once
        x = x.view(b * t, c, h, w)

        # Extract features using EfficientNet
        feats = self.backbone(x)  # Shape: (b*t, 1280)

        # Reshape back to sequence format
        feats = feats.view(b, t, -1)  # Shape: (b, t, 1280)

        # Process sequence with LSTM
        _, (h_n, _) = self.lstm(feats)  # h_n shape: (1, b, hidden_size)
        h_n = h_n.squeeze(0)  # Shape: (b, hidden_size)

        # Final classification
        out = self.dropout(h_n)
        out = self.fc(out)

        return torch.sigmoid(out)

print("Model class defined successfully!")

Model class defined successfully!


In [ ]:
class VideoProcessor:
    def __init__(self, img_size=224, max_seq_length=16):
        self.img_size = img_size
        self.max_seq_length = max_seq_length
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Image preprocessing pipeline
        self.transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])

    def extract_frames(self, video_path):
        """Extract frames from video file"""
        cap = cv2.VideoCapture(str(video_path))
        frames = []

        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        duration = total_frames / fps if fps > 0 else 0

        print(f"Video info: {total_frames} frames, {fps:.2f} FPS, {duration:.2f}s duration")

        frame_count = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Convert BGR to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame_rgb)
            frame_count += 1

            # Show progress for long videos
            if frame_count % 100 == 0:
                print(f"Processed {frame_count}/{total_frames} frames")

        cap.release()

        if len(frames) == 0:
            raise ValueError("No frames found in video")

        print(f"Extracted {len(frames)} frames")
        return frames

    def preprocess_frames(self, frames):
        """Preprocess frames for model input"""
        # Sample frames to match sequence length
        total_frames = len(frames)

        if total_frames >= self.max_seq_length:
            # Sample frames uniformly across the video
            indices = np.linspace(0, total_frames - 1, self.max_seq_length, dtype=int)
            selected_frames = [frames[i] for i in indices]
        else:
            # Pad with last frame if video is too short
            selected_frames = frames + [frames[-1]] * (self.max_seq_length - total_frames)

        # Convert to tensors and apply preprocessing
        processed_frames = []
        for frame in selected_frames:
            pil_frame = Image.fromarray(frame)
            tensor_frame = self.transform(pil_frame)
            processed_frames.append(tensor_frame)

        # Stack frames and add batch dimension
        video_tensor = torch.stack(processed_frames).unsqueeze(0)  # Shape: (1, seq_len, 3, H, W)

        return video_tensor.to(self.device)

    def create_frame_montage(self, frames, max_frames=8):
        """Create a montage of frames for visualization"""
        selected_frames = frames[::len(frames)//min(max_frames, len(frames))][:max_frames]

        fig, axes = plt.subplots(2, 4, figsize=(16, 8))
        axes = axes.flatten()

        for i, frame in enumerate(selected_frames):
            if i < len(axes):
                axes[i].imshow(frame)
                axes[i].set_title(f'Frame {i+1}')
                axes[i].axis('off')

        # Hide unused subplots
        for i in range(len(selected_frames), len(axes)):
            axes[i].axis('off')

        plt.tight_layout()
        plt.show()

print("Video processing functions defined successfully!")


Video processing functions defined successfully!


In [ ]:
# Video Processing Functions
class VideoProcessor:
    def __init__(self, img_size=224, max_seq_length=16):
        self.img_size = img_size
        self.max_seq_length = max_seq_length
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Image preprocessing pipeline
        self.transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])

    def extract_frames(self, video_path):
        """Extract frames from video file"""
        cap = cv2.VideoCapture(str(video_path))
        frames = []

        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        duration = total_frames / fps if fps > 0 else 0

        print(f"Video info: {total_frames} frames, {fps:.2f} FPS, {duration:.2f}s duration")

        frame_count = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Convert BGR to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame_rgb)
            frame_count += 1

            # Show progress for long videos
            if frame_count % 100 == 0:
                print(f"Processed {frame_count}/{total_frames} frames")

        cap.release()

        if len(frames) == 0:
            raise ValueError("No frames found in video")

        print(f"Extracted {len(frames)} frames")
        return frames

    def preprocess_frames(self, frames):
        """Preprocess frames for model input"""
        # Sample frames to match sequence length
        total_frames = len(frames)

        if total_frames >= self.max_seq_length:
            # Sample frames uniformly across the video
            indices = np.linspace(0, total_frames - 1, self.max_seq_length, dtype=int)
            selected_frames = [frames[i] for i in indices]
        else:
            # Pad with last frame if video is too short
            selected_frames = frames + [frames[-1]] * (self.max_seq_length - total_frames)

        # Convert to tensors and apply preprocessing
        processed_frames = []
        for frame in selected_frames:
            pil_frame = Image.fromarray(frame)
            tensor_frame = self.transform(pil_frame)
            processed_frames.append(tensor_frame)

        # Stack frames and add batch dimension
        video_tensor = torch.stack(processed_frames).unsqueeze(0)  # Shape: (1, seq_len, 3, H, W)

        return video_tensor.to(self.device)

    def create_frame_montage(self, frames, max_frames=8):
        """Create a montage of frames for visualization"""
        selected_frames = frames[::len(frames)//min(max_frames, len(frames))][:max_frames]

        fig, axes = plt.subplots(2, 4, figsize=(16, 8))
        axes = axes.flatten()

        for i, frame in enumerate(selected_frames):
            if i < len(axes):
                axes[i].imshow(frame)
                axes[i].set_title(f'Frame {i+1}')
                axes[i].axis('off')

        # Hide unused subplots
        for i in range(len(selected_frames), len(axes)):
            axes[i].axis('off')

        plt.tight_layout()
        plt.show()

print("Video processing functions defined successfully!")

Video processing functions defined successfully!


In [ ]:
# Model Upload and Loading Functions
def upload_model_from_device():
    """Interactive widget to upload model file from your device"""
    print("📤 UPLOAD YOUR TRAINED MODEL")
    print("="*40)

    model_upload = widgets.FileUpload(
        accept='.pth,.pt,.pkl,.bin',
        multiple=False,
        description='Upload Model',
        layout=Layout(width='300px')
    )

    upload_status = widgets.Output()

    def on_model_upload(change):
        with upload_status:
            clear_output()

            if model_upload.value:
                try:
                    # Get the uploaded file
                    filename = list(model_upload.value.keys())[0]
                    content = model_upload.value[filename]['content']

                    # Save to Colab filesystem
                    model_path = f"/content/{filename}"
                    with open(model_path, 'wb') as f:
                        f.write(content)

                    print(f"✅ Model uploaded successfully!")
                    print(f"📁 Saved as: {model_path}")
                    print(f"📊 File size: {len(content)/1024/1024:.1f} MB")
                    print(f"\n🔄 Now run: detector = ShopliftingDetector('{model_path}')")

                    # Store the path globally for easy access
                    global uploaded_model_path
                    uploaded_model_path = model_path

                except Exception as e:
                    print(f"❌ Error uploading model: {str(e)}")

    model_upload.observe(on_model_upload, names='value')

    display(widgets.VBox([
        widgets.HTML("<h4>🤖 Upload Your Trained Model</h4>"),
        widgets.HTML("<p>Select your trained model file (.pth, .pt, .pkl, .bin)</p>"),
        model_upload,
        upload_status
    ]))

def load_model_from_drive():
    """Load model from Google Drive"""
    print("💾 LOAD MODEL FROM GOOGLE DRIVE")
    print("="*40)

    try:
        from google.colab import drive
        drive.mount('/content/drive')

        print("✅ Google Drive mounted successfully!")
        print("📁 Your drive is available at: /content/drive/MyDrive/")
        print("\n📋 To use your model:")
        print("1. Upload your model file to Google Drive")
        print("2. Note the path (e.g., '/content/drive/MyDrive/models/shoplifter_model.pth')")
        print("3. Run: detector = ShopliftingDetector('your_model_path_here')")

        # List common model locations
        common_paths = [
            "/content/drive/MyDrive/shoplifter_model.pth",
            "/content/drive/MyDrive/models/shoplifter_model.pth",
            "/content/drive/MyDrive/ML_Models/shoplifter_model.pth"
        ]

        print("\n🔍 Checking common model locations...")
        for path in common_paths:
            if os.path.exists(path):
                print(f"✅ Found model at: {path}")
                global drive_model_path
                drive_model_path = path
                return path
            else:
                print(f"❌ Not found: {path}")

        print("\n💡 If your model is in a different location, use the full path")

    except Exception as e:
        print(f"❌ Error mounting Google Drive: {str(e)}")

    return None

def download_pretrained_model():
    """Download a pretrained model from URL (if you have one hosted)"""
    print("🌐 DOWNLOAD PRETRAINED MODEL")
    print("="*40)

    url_input = widgets.Text(
        placeholder='https://example.com/your_model.pth',
        description='Model URL:',
        layout=Layout(width='500px')
    )

    download_button = widgets.Button(
        description='📥 Download',
        button_style='success'
    )

    download_status = widgets.Output()

    def on_download_click(b):
        with download_status:
            clear_output()

            if not url_input.value:
                print("❌ Please enter a model URL")
                return

            try:
                import urllib.request

                print(f"📥 Downloading model from: {url_input.value}")

                # Download the model
                model_path = "/content/downloaded_model.pth"
                urllib.request.urlretrieve(url_input.value, model_path)

                # Check file size
                file_size = os.path.getsize(model_path) / 1024 / 1024
                print(f"✅ Model downloaded successfully!")
                print(f"📁 Saved as: {model_path}")
                print(f"📊 File size: {file_size:.1f} MB")
                print(f"\n🔄 Now run: detector = ShopliftingDetector('{model_path}')")

                global downloaded_model_path
                downloaded_model_path = model_path

            except Exception as e:
                print(f"❌ Error downloading model: {str(e)}")

    download_button.on_click(on_download_click)

    display(widgets.VBox([
        widgets.HTML("<h4>🌐 Download Model from URL</h4>"),
        widgets.HTML("<p>Enter the URL of your hosted model file</p>"),
        url_input,
        download_button,
        download_status
    ]))

class ShopliftingDetector:
    def __init__(self, model_path=None):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.processor = VideoProcessor()
        self.model = None

        # Initialize model
        self._initialize_model(model_path)

    def _initialize_model(self, model_path=None):
        """Initialize the model - either load pretrained or create new"""
        try:
            self.model = EfficientNetB0_LSTM()

            if model_path and os.path.exists(model_path):
                print(f"📂 Loading model from: {model_path}")

                # Load the state dict
                state_dict = torch.load(model_path, map_location=self.device)

                # Handle different model save formats
                if isinstance(state_dict, dict):
                    if 'model_state_dict' in state_dict:
                        # Model was saved with additional info
                        self.model.load_state_dict(state_dict['model_state_dict'])
                        if 'epoch' in state_dict:
                            print(f"📈 Model trained for {state_dict['epoch']} epochs")
                        if 'accuracy' in state_dict:
                            print(f"🎯 Training accuracy: {state_dict['accuracy']:.2%}")
                    else:
                        # Just the state dict
                        self.model.load_state_dict(state_dict)
                else:
                    # Assume it's the model itself
                    self.model = state_dict

                print(f"✅ Model loaded successfully!")
                print(f"🖥️  Running on: {self.device}")

                # Get model info
                total_params = sum(p.numel() for p in self.model.parameters())
                trainable_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
                print(f"🔢 Total parameters: {total_params:,}")
                print(f"🏋️ Trainable parameters: {trainable_params:,}")

            else:
                # Initialize with random weights (for demo purposes)
                print("⚠️  No model path provided or file doesn't exist")
                print("🎲 Using randomly initialized model (for demo only)")
                print("📋 To load your trained model:")
                print("   1. Use upload_model_from_device()")
                print("   2. Or use load_model_from_drive()")
                print("   3. Then create detector with: ShopliftingDetector('path_to_model')")

            self.model.to(self.device)
            self.model.eval()

        except Exception as e:
            print(f"❌ Error initializing model: {str(e)}")
            print("🔍 Common issues:")
            print("   - Model architecture doesn't match")
            print("   - Corrupt model file")
            print("   - Incompatible PyTorch version")
            self.model = None

    def load_new_model(self, model_path):
        """Load a new model without recreating the detector"""
        print(f"🔄 Loading new model: {model_path}")
        self._initialize_model(model_path)

        if self.model is not None:
            print("✅ Model updated successfully!")
        else:
            print("❌ Failed to load new model")

    def predict(self, video_path, threshold=0.5, show_frames=True):
        """Predict shoplifting behavior in video"""
        if self.model is None:
            raise RuntimeError("Model not initialized properly")

        print("🎬 Processing video...")
        start_time = time.time()

        try:
            # Extract and preprocess frames
            frames = self.processor.extract_frames(video_path)

            # Show frame montage
            if show_frames:
                print("\n📸 Sample frames from video:")
                self.processor.create_frame_montage(frames)

            # Preprocess for model
            video_tensor = self.processor.preprocess_frames(frames)

            # Make prediction
            print("\n🔍 Running inference...")
            with torch.no_grad():
                outputs = self.model(video_tensor)
                probability = outputs.squeeze().item()

            processing_time = time.time() - start_time

            # Determine final prediction
            is_shoplifter = probability > threshold
            prediction = "SHOPLIFTING DETECTED" if is_shoplifter else "NORMAL BEHAVIOR"

            # Display results
            self._display_results(prediction, probability, processing_time, is_shoplifter)

            return {
                'prediction': prediction,
                'probability': probability,
                'confidence': probability if is_shoplifter else (1 - probability),
                'processing_time': processing_time,
                'is_shoplifter': is_shoplifter
            }

        except Exception as e:
            print(f"❌ Error during prediction: {str(e)}")
            raise

    def _display_results(self, prediction, probability, processing_time, is_shoplifter):
        """Display formatted results"""
        print("\n" + "="*60)
        print("🤖 SHOPLIFTING DETECTION RESULTS")
        print("="*60)

        # Status with emoji
        status_emoji = "🚨" if is_shoplifter else "✅"
        print(f"{status_emoji} STATUS: {prediction}")

        # Probability bar
        print(f"\n📊 CONFIDENCE: {probability:.1%}")
        bar_length = 40
        filled_length = int(probability * bar_length)
        bar_color = "🟥" if is_shoplifter else "🟩"
        empty_color = "⬜"

        bar = bar_color * filled_length + empty_color * (bar_length - filled_length)
        print(f"    [{bar}] {probability:.3f}")

        # Processing time
        print(f"\n⏱️  PROCESSING TIME: {processing_time:.2f} seconds")

        # Risk assessment
        if is_shoplifter:
            if probability > 0.8:
                risk_level = "HIGH RISK ⚠️"
            elif probability > 0.6:
                risk_level = "MEDIUM RISK ⚠️"
            else:
                risk_level = "LOW RISK ⚠️"
        else:
            risk_level = "NO RISK ✅"

        print(f"🎯 RISK LEVEL: {risk_level}")
        print("="*60)


In [ ]:
# Model Upload Interface

print("🤖 MODEL LOADING OPTIONS")
print("="*50)

# Display all model loading options
print("📋 Choose how to load your trained model:")
print("\n1️⃣ UPLOAD FROM YOUR DEVICE:")
uploaded_model_path = upload_model_from_device()

print("\n" + "="*50)
print("2️⃣ LOAD FROM GOOGLE DRIVE:")
drive_model_path = load_model_from_drive()

print("\n" + "="*50)
print("3️⃣ DOWNLOAD FROM URL:")
downloaded_model_path = download_pretrained_model()

print("\n" + "="*50)
print("📝 USAGE EXAMPLES:")
print("# After uploading, create detector with your model:")
print("detector = ShopliftingDetector('/content/your_model.pth')")
print("# Or use the automatically saved path:")
print("detector = ShopliftingDetector(uploaded_model_path)  # if uploaded")
print("detector = ShopliftingDetector(drive_model_path)     # if from drive")

#------------------------------------------------
# NEW LOGIC TO AUTO-DETECT THE MODEL

selected_model_path = None

if uploaded_model_path and os.path.exists(uploaded_model_path):
    selected_model_path = uploaded_model_path
elif drive_model_path and os.path.exists(drive_model_path):
    selected_model_path = drive_model_path
elif downloaded_model_path and os.path.exists(downloaded_model_path):
    selected_model_path = downloaded_model_path

# Initialize detector
print("\n🚀 Initializing detector...")
if selected_model_path:
    detector = ShopliftingDetector(selected_model_path)
    print(f"✅ Detector ready! Loaded model: {selected_model_path}")
else:
    detector = ShopliftingDetector()  # fallback to demo model
    print("⚠️ No trained model found. Using demo model instead.")


🤖 MODEL LOADING OPTIONS
📋 Choose how to load your trained model:

1️⃣ UPLOAD FROM YOUR DEVICE:
📤 UPLOAD YOUR TRAINED MODEL



2️⃣ LOAD FROM GOOGLE DRIVE:
💾 LOAD MODEL FROM GOOGLE DRIVE
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive mounted successfully!
📁 Your drive is available at: /content/drive/MyDrive/

📋 To use your model:
1. Upload your model file to Google Drive
2. Note the path (e.g., '/content/drive/MyDrive/models/shoplifter_model.pth')
3. Run: detector = ShopliftingDetector('your_model_path_here')

🔍 Checking common model locations...
✅ Found model at: /content/drive/MyDrive/shoplifter_model.pth

3️⃣ DOWNLOAD FROM URL:
🌐 DOWNLOAD PRETRAINED MODEL



📝 USAGE EXAMPLES:
# After uploading, create detector with your model:
detector = ShopliftingDetector('/content/your_model.pth')
# Or use the automatically saved path:
detector = ShopliftingDetector(uploaded_model_path)  # if uploaded
detector = ShopliftingDetector(drive_model_path)     # if from drive

🚀 Initializing detector...
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 20.5M/20.5M [00:00<00:00, 133MB/s] 


📂 Loading model from: /content/drive/MyDrive/shoplifter_model.pth
✅ Model loaded successfully!
🖥️  Running on: cuda
🔢 Total parameters: 4,729,597
🏋️ Trainable parameters: 722,049
✅ Detector ready! Loaded model: /content/drive/MyDrive/shoplifter_model.pth


In [ ]:
# CELL 7: File Upload Widget for Videos

def create_upload_interface():
    """Create an interactive upload interface"""

    # File upload widget
    upload_widget = widgets.FileUpload(
        accept='video/*',
        multiple=False,
        description='Upload Video',
        layout=Layout(width='300px')
    )

    # Analysis button
    analyze_button = widgets.Button(
        description='🔍 Analyze Video',
        button_style='primary',
        layout=Layout(width='200px', height='40px')
    )

    # Results area
    results_output = widgets.Output()

    def on_upload_change(change):
        if upload_widget.value:
            filename = list(upload_widget.value.keys())[0]
            analyze_button.description = f'🔍 Analyze: {filename[:20]}...'
        else:
            analyze_button.description = '🔍 Analyze Video'

    def on_analyze_click(b):
        with results_output:
            clear_output()

            if not upload_widget.value:
                print("❌ Please upload a video file first!")
                return

            try:
                # Get uploaded file
                filename = list(upload_widget.value.keys())[0]
                content = upload_widget.value[filename]['content']

                # Save temporarily
                temp_path = f"/tmp/{filename}"
                with open(temp_path, 'wb') as f:
                    f.write(content)

                print(f"📁 Processing: {filename}")
                print(f"📊 File size: {len(content)/1024/1024:.1f} MB")

                # Run analysis
                result = detector.predict(temp_path)

                # Cleanup
                os.remove(temp_path)

            except Exception as e:
                print(f"❌ Error: {str(e)}")

    # Connect event handlers
    upload_widget.observe(on_upload_change, names='value')
    analyze_button.on_click(on_analyze_click)

    # Layout
    upload_box = widgets.VBox([
        widgets.HTML("<h3>🎬 Shoplifting Detection System</h3>"),
        widgets.HTML("<p>Upload a video file to analyze for potential shoplifting behavior.</p>"),
        upload_widget,
        analyze_button,
        results_output
    ])

    return upload_box

# Display the interface
interface = create_upload_interface()
display(interface)

In [ ]:
# Sample Video Analysis (Optional)
def analyze_sample_video():
    """Analyze a sample video if available"""

    # Check for sample videos in common paths
    sample_paths = [
        "/content/sample_video.mp4",
        "/content/drive/MyDrive/sample_video.mp4",
        "sample_video.mp4"
    ]

    sample_path = None
    for path in sample_paths:
        if os.path.exists(path):
            sample_path = path
            break

    if sample_path:
        print(f"🎬 Found sample video: {sample_path}")
        print("Running analysis...")

        try:
            result = detector.predict(sample_path)
            return result
        except Exception as e:
            print(f"❌ Error analyzing sample video: {str(e)}")
    else:
        print("ℹ️  No sample video found. Upload a video using the interface above.")
        print("\n📝 To test with your own video:")
        print("1. Upload a video file using the widget above, OR")
        print("2. Save a video file to /content/sample_video.mp4 and run this cell again")

# Run sample analysis
# analyze_sample_video()


In [ ]:
# API-style Function for Batch Processing

def batch_analyze_videos(video_paths, output_file="results.txt"):
    """Analyze multiple videos and save results"""
    results = []

    print(f"🔄 Processing {len(video_paths)} videos...")

    for i, video_path in enumerate(video_paths, 1):
        print(f"\n📹 Processing video {i}/{len(video_paths)}: {video_path}")

        try:
            result = detector.predict(video_path, show_frames=False)
            result['video_path'] = video_path
            results.append(result)

            # Print summary
            status = "🚨 SUSPICIOUS" if result['is_shoplifter'] else "✅ NORMAL"
            print(f"   Result: {status} (confidence: {result['confidence']:.1%})")

        except Exception as e:
            print(f"   ❌ Error: {str(e)}")
            results.append({
                'video_path': video_path,
                'error': str(e)
            })

    # Save results
    with open(output_file, 'w') as f:
        f.write("Shoplifting Detection Results\n")
        f.write("="*50 + "\n\n")

        for result in results:
            f.write(f"Video: {result['video_path']}\n")
            if 'error' in result:
                f.write(f"Status: ERROR - {result['error']}\n")
            else:
                status = "SUSPICIOUS" if result['is_shoplifter'] else "NORMAL"
                f.write(f"Status: {status}\n")
                f.write(f"Confidence: {result['confidence']:.1%}\n")
                f.write(f"Processing Time: {result['processing_time']:.2f}s\n")
            f.write("-" * 30 + "\n")

    print(f"\n📄 Results saved to: {output_file}")
    return results

print("✅ Batch processing function ready!")
print("\n🎯 USAGE EXAMPLES:")
print("1. Use the upload widget above for interactive analysis")
print("2. For batch processing: batch_analyze_videos(['video1.mp4', 'video2.mp4'])")
print("3. For single video: detector.predict('path/to/video.mp4')")

✅ Batch processing function ready!

🎯 USAGE EXAMPLES:
1. Use the upload widget above for interactive analysis
2. For batch processing: batch_analyze_videos(['video1.mp4', 'video2.mp4'])
3. For single video: detector.predict('path/to/video.mp4')


In [ ]:
# Model Training Setup (Optional)
def setup_training():
    """Setup for training the model (if you have training data)"""

    print("🏋️ Training Setup Information")
    print("="*40)
    print("To train this model, you would need:")
    print("1. Video dataset with labeled shoplifting/normal behaviors")
    print("2. Data preprocessing pipeline")
    print("3. Training loop with loss function and optimizer")
    print("4. Validation and testing procedures")
    print("\n📚 Training would typically involve:")
    print("- Binary cross-entropy loss")
    print("- Adam optimizer")
    print("- Learning rate scheduling")
    print("- Data augmentation for videos")
    print("- K-fold cross-validation")

    # Example training setup (skeleton)
    def train_model(train_loader, val_loader, num_epochs=10):
        """Skeleton for training function"""
        model = EfficientNetB0_LSTM()
        criterion = nn.BCELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

        print("🚀 Training would start here...")
        print("(This is a skeleton - implement with your training data)")

        return model

    print("\n💡 For a complete training implementation, you would need:")
    print("   - Dataset class for video loading")
    print("   - Training and validation loops")
    print("   - Model checkpointing")
    print("   - Performance metrics calculation")

setup_training()



🏋️ Training Setup Information
To train this model, you would need:
1. Video dataset with labeled shoplifting/normal behaviors
2. Data preprocessing pipeline
3. Training loop with loss function and optimizer
4. Validation and testing procedures

📚 Training would typically involve:
- Binary cross-entropy loss
- Adam optimizer
- Learning rate scheduling
- Data augmentation for videos
- K-fold cross-validation

💡 For a complete training implementation, you would need:
   - Dataset class for video loading
   - Training and validation loops
   - Model checkpointing
   - Performance metrics calculation


In [ ]:
# FINAL INSTRUCTIONS

print("\n" + "="*60)
print("🎉 SHOPLIFTING DETECTION SYSTEM READY!")
print("="*60)
print("\n📋 QUICK START GUIDE:")
print("1. ⬆️  Use the upload widget above to select a video file")
print("2. 🔍 Click 'Analyze Video' to process the video")
print("3. 📊 View the detailed results and confidence scores")
print("4. 🔄 Repeat with different videos as needed")
print("\n⚠️  IMPORTANT NOTES:")
print("- This is a demo version with a randomly initialized model")
print("- For production use, you need a properly trained model")
print("- Results are for demonstration purposes only")
print("- Consider ethical implications of surveillance systems")
print("\n🛠️  ADVANCED USAGE:")
print("# Update model after initialization:")
print("detector.load_new_model('/path/to/new/model.pth')")
print("\n# Check model status:")
print("if detector.model is not None:")
print("    print('Model loaded and ready!')")
print("else:")
print("    print('Please load a model first')")


🎉 SHOPLIFTING DETECTION SYSTEM READY!

📋 QUICK START GUIDE:
1. ⬆️  Use the upload widget above to select a video file
2. 🔍 Click 'Analyze Video' to process the video
3. 📊 View the detailed results and confidence scores
4. 🔄 Repeat with different videos as needed

⚠️  IMPORTANT NOTES:
- This is a demo version with a randomly initialized model
- For production use, you need a properly trained model
- Results are for demonstration purposes only
- Consider ethical implications of surveillance systems

🛠️  ADVANCED USAGE:
# Update model after initialization:
detector.load_new_model('/path/to/new/model.pth')

# Check model status:
if detector.model is not None:
    print('Model loaded and ready!')
else:
    print('Please load a model first')
